# Comparing the Naselle USGS rainfall gauge with PRISM gridded rainfall

In [1]:
%matplotlib widget

import __init__
import scripts.config as config
import pandas as pd
import matplotlib.pyplot as plt
import mpld3
import numpy as np

In [2]:
XSMALL_SIZE = 6
SMALL_SIZE = 7
MEDIUM_SIZE = 9
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 140

In [3]:
gauge = pd.read_csv(config.daily_ppt.parents[0] / 'GHCND_USC00455774_1929_2020.csv', parse_dates=True, index_col=5)
gauge['SNOW'].fillna(0, inplace=True)
gauge['SNOW_SWE'] = gauge['SNOW'] / 13
gauge['PRCP_TOT'] = gauge['PRCP'] + gauge['SNOW_SWE']
gauge_precip = gauge[['PRCP_TOT']]

prism_precip = pd.read_csv(str(config.daily_ppt), parse_dates=True, index_col=0)

# Expand precip record to full date range in case some days are missing
velma_start = pd.to_datetime('01-01-2003')
velma_end = pd.to_datetime('12-31-2019')
rng = pd.date_range(velma_start, velma_end)
date_df = pd.DataFrame(index=rng)
gauge_precip_velma = date_df.merge(gauge_precip, left_index=True, right_index=True, how='left')
prism_precip_velma = date_df.merge(prism_precip, left_index=True, right_index=True, how='left')

# Subset to simulation date range and add doy and year columns
start = pd.to_datetime('2003-01-01')
end = pd.to_datetime('2007-12-31')
gauge_precip04_07 = gauge_precip[(gauge_precip.index >= start) & (gauge_precip.index <= end)].copy()
prism_precip04_07 = prism_precip[(prism_precip.index >= start) & (prism_precip.index <= end)].copy()

gauge_precip04_07['doy'], gauge_precip04_07['year'] = gauge_precip04_07.index.dayofyear, gauge_precip04_07.index.year
prism_precip04_07['doy'], prism_precip04_07['year'] = prism_precip04_07.index.dayofyear, prism_precip04_07.index.year

C:\Users\ipdavies\AppData\Local\Continuum\anaconda3\envs\tnc_velma\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (9,13,37,43,45) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# PRISM vs. nearby gauge precipitation
gauge_yearly = pd.pivot_table(gauge_precip04_07, index=['doy'], columns=['year'], values=['PRCP_TOT'])
prism_yearly = pd.pivot_table(prism_precip04_07, index=['doy'], columns=['year'], values=['mean_ppt_mm'])

years = gauge_yearly.columns.get_level_values(1)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
for col, year in enumerate(years):
    gauge_yearly.iloc[:, col].plot(ax=axes[col], label='Gauge', linewidth=0.6)
    prism_yearly.iloc[:, col].plot(ax=axes[col], label='PRISM', linewidth=0.6)
    axes[col].set_title(year)
    axes[col].set_ylim([0, 80])
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.3), fancybox=True, ncol=2)
axes[0].set_ylabel('Precipitation (mm)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Looks like the peaks are misaligned by a day - roll gauge values back by a day and compare with PRISM
gauge_precip04_07['PRCP_TOT_tmin1'] = gauge_precip04_07['PRCP_TOT'].shift(-1)
gauge_yearly_tmin1 = pd.pivot_table(gauge_precip04_07, index=['doy'], columns=['year'], values=['PRCP_TOT_tmin1'])

years = gauge_yearly.columns.get_level_values(1)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
for col, year in enumerate(years):
    gauge_yearly_tmin1.iloc[:, col].plot(ax=axes[col], label='Gauge (t-1)', linewidth=0.6)
    prism_yearly.iloc[:, col].plot(ax=axes[col], label='PRISM', linewidth=0.6)
    axes[col].set_title(year)
    axes[col].set_ylim([0, 80])
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.3), fancybox=True, ncol=2)
axes[0].set_ylabel('Precipitation (mm)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# Comparing observed runoff with gauge precip and gauge precip(t-1)
runoff_obs = pd.read_csv(config.velma_data / 'runoff' / 'ellsworth_Q_2003_2007_dummy.csv', names=['runoff_obs'])
runoff_obs.index = pd.date_range('01-01-2003', '12-31-2007')
runoff_obs['doy'], runoff_obs['year'] = runoff_obs.index.dayofyear, runoff_obs.index.year
runoff_obs_yearly = pd.pivot_table(runoff_obs, index=['doy'], columns=['year'], values=['runoff_obs'])

years = gauge_yearly.columns.get_level_values(1)
fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
for col, year in enumerate(years):
    gauge_yearly_tmin1.iloc[:, col].plot(ax=axes[col], label='Gauge (t-1)', linewidth=0.6)
    gauge_yearly.iloc[:, col].plot(ax=axes[col], label='Gauge', linewidth=0.6)
    runoff_obs_yearly.iloc[:, col].plot(ax=axes[col], label='Runoff', linewidth=0.6)
    axes[col].set_title(year)
    axes[col].set_ylim([0, 80])
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.5), fancybox=True, ncol=3)
axes[0].set_ylabel('Precipitation (mm)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Fill missing gauge dates with PRISM data
gauge_precip_velma['PRCP_TOT'] = gauge_precip_velma['PRCP_TOT'].combine_first(prism_precip_velma['mean_ppt_mm'])

In [8]:
# # Export t-1 gauge data (shifted back a day)

# gauge_precip_tmin1 = gauge_precip.copy()
# gauge_precip_tmin1['PRCP_tmin1'] = gauge_precip_tmin1['PRCP_TOT'].shift(-1)

# start = pd.to_datetime('2003-01-01')
# end = pd.to_datetime('2007-12-31')
# gauge_precip_tmin1_velma = gauge_precip_tmin1[(gauge_precip_tmin1.index >= start) & (gauge_precip_tmin1.index <= end)]['PRCP_tmin1'].copy()

# outfile = str(config.velma_data / 'precip' / 'GHCND_USC00455774_ppt_tmin1_2003_2019.csv')
# gauge_precip_tmin1_velma.to_csv(outfile, header=False, index=False)

In [9]:
# # Export gauge data
# outfile = str(config.velma_data / 'precip' / 'GHCND_USC00455774_ppt_2003_2019.csv')
# if len(pd.date_range(velma_start, velma_end)) != len(gauge_precip_velma):
#     print('STOP: Duplicates/missing values exist in output file: ', outfile)
# gauge_precip_velma.to_csv(outfile, header=False, index=False)

In [9]:
# # Export averaged PRISM and gauge data
# avg = np.mean([gauge_precip_velma, prism_precip_velma], axis=0)
# avg_precip_velma = pd.DataFrame(data=avg, columns=['avg_ppt'])
# outfile = str(config.velma_data / 'precip' / 'PRISM_gauge_avg_ppt_2003_2019.csv')
# if len(pd.date_range(velma_start, velma_end)) != len(avg_precip_velma):
#     print('STOP: Duplicates/missing values exist in output file: ', outfile)
# avg_precip_velma.to_csv(outfile, header=False, index=False)

In [9]:
# # Compare prism and exported naselle precip measurements to make sure they are different

# ppt1 = pd.read_csv(outfile, names=['ppt1'])
# ppt2 = pd.read_csv(str(config.velma_data / 'precip' / 'ellsworth_ppt_2003_2019.csv'), names=['ppt2'])

# velma_start = pd.to_datetime('01-01-2003')
# velma_end = pd.to_datetime('12-31-2019')
# rng = pd.date_range(velma_start, velma_end)
# date_df = pd.DataFrame(index=rng)
# ppt1.index = rng
# ppt2.index = rng

# ppt1['doy'], ppt1['year'] = ppt1.index.dayofyear, ppt1.index.year
# ppt2['doy'], ppt2['year'] = ppt2.index.dayofyear, ppt2.index.year

# start = pd.to_datetime('2003-01-01')
# end = pd.to_datetime('2007-12-31')
# ppt1 = ppt1[(ppt1.index >= start) & (ppt1.index <= end)].copy()
# ppt2 = ppt2[(ppt2.index >= start) & (ppt2.index <= end)].copy()

# ppt1_yearly = pd.pivot_table(ppt1, index=['doy'], columns=['year'], values=['ppt1'])
# ppt2_yearly = pd.pivot_table(ppt2, index=['doy'], columns=['year'], values=['ppt2'])


# years = ppt1_yearly.columns.get_level_values(1)
# fig, axes = plt.subplots(ncols=1, nrows=len(years), figsize=(6, 9))
# for col, year in enumerate(years):
#     ppt1_yearly.iloc[:, col].plot(ax=axes[col], label='Gauge', linewidth=0.6)
#     ppt2_yearly.iloc[:, col].plot(ax=axes[col], label='PRISM', linewidth=0.6)
#     axes[col].set_title(year)
#     axes[col].set_ylim([0, 80])
# axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1.3), fancybox=True, ncol=2)
# plt.tight_layout()

ValueError: Length mismatch: Expected axis has 1461 elements, new values have 5844 elements

Using precip/temp downloaded from PRISM data explorer page